In [ ]:
!pip install torch
!pip install transformers
!nltk.download("wordnet")
!pip install jedi

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, RobertaTokenizer, T5ForConditionalGeneration
import torch
from nltk.corpus import wordnet as wn
import nltk


path = "eclipse_platform.csv"

df = pd.read_csv(path)

## Prepare the models

## Predict functions

In [ ]:
# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
model = AutoModelForSequenceClassification.from_pretrained("papluca/xlm-roberta-base-language-detection")

def detect_language(text):
    # Encode the text
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    # Get the logits for the text
    logits = model(input_ids)[0]
    # Get the class probabilities
    probs = torch.nn.functional.softmax(logits, dim=1)
    # Get the class label
    label_id = torch.argmax(probs, dim=1).item()

    return label_id, probs[0][label_id].item()

In [ ]:
# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

def sentiment_analysis(text):
    # Encode the text
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    # Get the logits for the text
    logits = model(input_ids)[0]
    # Get the class probabilities
    probs = torch.nn.functional.softmax(logits, dim=1)
    # Get the class label
    label_id = torch.argmax(probs, dim=1).item()

    if label_id == 0:
      sentiment = "negative"
    elif label_id == 1:
      sentiment = "neutral"
    else:
      sentiment = "positive"

    return sentiment, probs[0][label_id].item()
  

## Utils functions

In [ ]:
def check_case(text):
    camel_case = r'[a-z][A-Z][a-z]*'
    snake_case = r'[a-z]+_[a-z]+'

    if re.search(camel_case, text):
        return True
    elif re.search(snake_case, text):
        return True
    else:
        return False

In [ ]:
def check_keywords(text):
  keywords = open("drive/MyDrive/Tesi/keywords.txt", "r").readlines()

  to_analyze = text.split()

  count = 0
  for keyword in keywords:
    count += to_analyze.count(keyword.replace("\n",""))

  return count

In [ ]:
def check_symbols(text):
  symbols = ["{","}","(",")",";","[","]","<",">","="]

  count = 0
  for symbol in symbols:
    count += text.count(symbol)
  
  return count

In [ ]:
def check_punctuation(text):
  count = 0
  count += text.count(". ")
  count += text.count(", ")
  count += text.count("! ")
  count += text.count("? ")

  return count

In [ ]:
def percentage_of_spaces(text):
  spaces = text.count(" ")
  chars = len(text)

  return spaces/chars

In [ ]:
def score_to_words(score, text):
  words = len(text.split())

  return score/words

## Select the bug report for example

In [ ]:
# 1642 1776 60 85 857
bug_rep_ex = df.loc[df["Issue_id"] == 4615, "Description"].values[0]

bug_rep_ex = bug_rep_ex.replace(";;","^").replace("\t","")
bug_rep_ex

## Function

In [ ]:
def CLASSIFIER(bug_report,len_min_string_code,
perc_len_min_string_code,perc_detect_language,
point_check_case,perc_spaces,point_perc_of_spaces,
point_sentiment,perc_score_to_words):
score = 0
  if len(bug_report) > 0:
    if len(bug_report) < len_min_string_code and check_symbols(bug_report) >= perc_len_min_string_code * len(bug_report) or detect_language(bug_report)[1] < perc_detect_language:
      score += check_keywords(bug_report)
      score += check_symbols(bug_report)
      score -= check_punctuation(bug_report)
      
      if check_case(bug_report):
        score += point_check_case
      
      if percentage_of_spaces(bug_report) > perc_spaces:
        score -= point_perc_of_spaces
      
      if sentiment_analysis(bug_report)[0] == "positive":
        score -= point_sentiment
      
      return score_to_words(score, bug_report) > perc_score_to_words
      
  return False